In [1]:
import pandas as pd
import numpy as np

Xtr = pd.read_csv('data/Xtr_tsvd.csv')
Xte = pd.read_csv('data/Xte_tsvd.csv')
y = pd.read_csv('data/Ytr1.csv')

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(Xtr,y['total'], test_size = 0.2)

In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

Xtr1 = scaler.fit_transform(Xtr)
Xte1 = scaler.transform(Xte)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [8]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 5, random_state = 42, n_jobs = -1)
rf_random.fit(X_train,y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42)

In [10]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy
base_model = RandomForestRegressor(n_estimators = 10, random_state = 42)
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_test, y_test)

Model Performance
Average Error: 11.5612 degrees.
Accuracy = -inf%.


In [11]:
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_test, y_test)

Model Performance
Average Error: 11.7090 degrees.
Accuracy = -inf%.


In [12]:
rf_random.best_params_

{'n_estimators': 1800,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 30,
 'bootstrap': False}

In [14]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True,False],
    'max_depth': [20,30,40],
    'min_samples_leaf': [1, 2, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [1600,1800,2000,2200]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [15]:
grid_search.fit(X_train,y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits


GridSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'bootstrap': [True, False], 'max_depth': [20, 30, 40],
                         'min_samples_leaf': [1, 2, 5],
                         'min_samples_split': [8, 10, 12],
                         'n_estimators': [1600, 1800, 2000, 2200]},
             verbose=2)

In [16]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 40,
 'min_samples_leaf': 5,
 'min_samples_split': 8,
 'n_estimators': 1800}

In [21]:
rf = RandomForestRegressor(bootstrap = True,max_depth = 40,n_estimators = 1800, min_samples_leaf = 5, min_samples_split = 8, n_jobs = -1)

rf.fit(Xtr1,y['total'])
(rf.predict(Xte1)-max(rf.predict(Xte1)))/(max(rf.predict(Xte1)) - min(rf.predict(Xte1)))

array([67.71540928,  5.42217186,  8.21539416, ..., 17.04638305,
        0.41090825,  8.71317974])

In [22]:
(rf.predict(Xte1)-max(rf.predict(Xte1)))/(max(rf.predict(Xte1)) - min(rf.predict(Xte1)))

array([-0.84284329, -0.98743914, -0.98095548, ..., -0.96045687,
       -0.99907135, -0.97980001])